In [187]:
import pandas as pd
import re

In [188]:
def get_graph(file_ame):
    f = open(file_ame, 'r')
    line = f.readline()
    graph = pd.DataFrame({'head_node':[], 'tail_node':[], 'distance':[]})
    while line:
        splited_line = re.split('\t| |,',line.strip())
        head = int(splited_line[0])
        splited_line.remove(splited_line[0])
        for pair in range(int(len(splited_line)/2)):
            tail = int(splited_line[pair*2])
            distance = int(splited_line[pair*2+1])
            graph = graph.append(pd.DataFrame({'head_node':[head], 'tail_node':[tail], 'distance':[distance]}))
        line = f.readline()
    graph.reset_index(inplace= True)
    graph.drop('index', axis=1, inplace= True)
    return graph

In [189]:
#graph =  get_graph('simplegraph.txt')
graph =  get_graph('data.txt')

In [190]:
def bfs(graph, starting_node):
    reachable_nodes = []
    start_node=1
    subset_graph = graph.loc[graph['head_node']==start_node]
    tail_nodes = subset_graph.groupby("tail_node", as_index=False)["distance"].min()['tail_node'].tolist()
    additions = [x for x in tail_nodes if x not in reachable_nodes]
    while len(additions)>0:
        reachable_nodes = reachable_nodes + additions
        subset_graph = graph.loc[graph['head_node'].isin(additions)]
        tail_nodes = subset_graph.groupby("tail_node", as_index=False)["distance"].min()['tail_node'].tolist()
        additions = [x for x in tail_nodes if x not in reachable_nodes]
    return sorted(reachable_nodes)
reachable_nodes = bfs(graph, starting_node=1)


In [193]:
def dijkstras(graph, starting_node):
    unexplored_nodes = bfs(graph, starting_node)+[starting_node]
    explored_nodes = []
    shortest_distance={}
    node_heap = {starting_node:0}
    while len(unexplored_nodes)>0:
        next_node_to_explore = min(node_heap, key=node_heap.get)
        next_node_val_currently = node_heap.pop(next_node_to_explore,0)
        shortest_distance[next_node_to_explore] = next_node_val_currently
        unexplored_nodes.remove(next_node_to_explore)
        explored_nodes.append(next_node_to_explore)
        subset_graph = graph.loc[(graph.head_node==next_node_to_explore) & (graph.tail_node.isin(unexplored_nodes))]
        subset_graph_minimized_unique = subset_graph.groupby("tail_node", as_index=False)["distance"].min()
        subset_graph_minimized_unique.distance += next_node_val_currently
        for node in range(len(subset_graph_minimized_unique)):
            tail = subset_graph_minimized_unique.loc[node][0] 
            if tail not in node_heap:
                node_heap[subset_graph_minimized_unique.loc[node][0]]=subset_graph_minimized_unique.loc[node][1]
            elif tail in node_heap:
                current_val_in_heap = node_heap[tail]
                if subset_graph_minimized_unique.loc[node][1] < current_val_in_heap:
                    node_heap[subset_graph_minimized_unique.loc[node][0]]=subset_graph_minimized_unique.loc[node][1]
                
    return shortest_distance

In [194]:
shortest_distance = dijkstras(graph, 1)

In [195]:
interested_nodes = [7, 37, 59, 82, 99, 115, 133, 165, 188, 197]
result=''
for nodes in interested_nodes:
    result += str(shortest_distance[nodes])+','
print (result)

2599.0,2610.0,2947.0,2052.0,2367.0,2399.0,2029.0,2442.0,2505.0,3068.0,
